In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression , RidgeClassifier, Huber
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier , ExtraTreesClassifier , AdaBoostClassifier
from sklearn.model_selection import KFold ,StratifiedKFold
from sklearn.metrics import log_loss
import warnings 
warnings.filterwarnings('ignore')

In [4]:
train = train[train['time']<2*1e6]
train = train[train['time']!=1804691]
train = train[train['Store_Ratio']>0]
train.head()

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,time,popularity
0,0.407,0.00380,2,-35.865,1,0.9920,0.944,0.0988,0.1100,113.911,189125,4
1,0.234,0.10500,0,-19.884,1,0.9440,0.900,0.1290,0.0382,76.332,186513,4
2,0.668,0.72600,9,-7.512,1,0.0387,0.000,0.1340,0.4530,124.075,172143,4
3,0.184,0.00561,4,-34.357,1,0.8490,0.931,0.1110,0.0641,79.037,184000,5
4,0.231,0.13100,6,-22.842,1,0.9370,0.000,0.1090,0.0677,109.560,186507,3


In [5]:
test['Store_Score'] = test['Store_Score'].apply(lambda x: -1*x if x>0 else x)

In [6]:
x = test[test['Store_Score']>0]
x.index,x.Store_Score.values

(Int64Index([], dtype='int64'), array([], dtype=float64))

In [7]:
x = train[train['Store_Ratio']>=0.99]
x.index

Int64Index([8096], dtype='int64')

In [8]:
train.drop(8096,axis=0,inplace=True)

In [9]:
train.popularity.value_counts()

4    15229
3     2165
5      472
1      322
Name: popularity, dtype: int64

In [10]:
train = train[train['Store_Presence']>0]

In [11]:
df = train.append(test)

In [12]:
df['time']  = df['time']/3600
df.skew()

Store_Ratio      -0.456594
Basket_Ratio     -0.128571
Category_1        0.045071
Store_Score      -1.154776
Category_2       -0.611812
Store_Presence    0.115197
Score_1           0.626505
Score_2           2.882750
Score_3           0.274964
Score_4           0.408695
time              3.779856
popularity       -2.949265
dtype: float64

In [13]:
df['Score_2'] = np.log1p(df['Score_2'])
df.skew()

Store_Ratio      -0.456594
Basket_Ratio     -0.128571
Category_1        0.045071
Store_Score      -1.154776
Category_2       -0.611812
Store_Presence    0.115197
Score_1           0.626505
Score_2           2.382999
Score_3           0.274964
Score_4           0.408695
time              3.779856
popularity       -2.949265
dtype: float64

In [14]:
df['Store_Pre_Bin'] = df['Store_Presence'].apply(lambda x: 1 if x<=0.5 else 0)
df['avg_score'] = (df['Score_1']+df['Score_2']+df['Score_3']+df['Score_4'])/4 
df['score_by_presence'] = df['avg_score']/df['Store_Presence']
df['score_by_basket'] = df['avg_score']/df['Basket_Ratio']

In [15]:
df['avg_score'] = (df['Score_1']+df['Score_2']+df['Score_3']+df['Score_4'])/4 
df['score_by_presence'] = df['avg_score']/df['Store_Presence']
df['score_by_basket'] = df['avg_score']/df['Basket_Ratio']

In [16]:
df['Basket_Ratio_sum_Store_Ratio'] = df['Basket_Ratio'] + df['Store_Ratio']
df['Store_Ratio_ratio_Store_Presence'] = df['Store_Ratio']/df['Store_Presence']
df['Score_3_ratio_Store_Ratio'] = df['Score_3']/df['Store_Ratio']
df['Score_3_multiply_Store_Ratio'] = df['Score_3']*df['Store_Ratio']
df['Score_3_ratio_Store_Ratio'] = df['Score_3']/df['Store_Ratio']
df['Store_Ratio_diff_Store_Presence'] = df['Store_Ratio']-df['Store_Presence']


In [17]:
df.columns

Index(['Store_Ratio', 'Basket_Ratio', 'Category_1', 'Store_Score',
       'Category_2', 'Store_Presence', 'Score_1', 'Score_2', 'Score_3',
       'Score_4', 'time', 'popularity', 'Store_Pre_Bin', 'avg_score',
       'score_by_presence', 'score_by_basket', 'Basket_Ratio_sum_Store_Ratio',
       'Store_Ratio_ratio_Store_Presence', 'Score_3_ratio_Store_Ratio',
       'Score_3_multiply_Store_Ratio', 'Store_Ratio_diff_Store_Presence'],
      dtype='object')

In [18]:
train,test = df.head(len(train)),df.tail(len(test))
test.drop(columns=['popularity'],axis=1,inplace=True)

In [19]:
X = train.drop('popularity',axis=1)
Y = train['popularity']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_ = scaler.fit_transform(X)
X = pd.DataFrame(X_,columns=X.columns)


In [28]:
folds = StratifiedKFold(n_splits=10,shuffle=True ,random_state=42)

oof_preds = []
oof_scores = []

for i , (train_idx,test_idx) in enumerate(folds.split(X,Y)):
  train_set = (X.iloc[train_idx],Y.iloc[train_idx])
  test_set = (X.iloc[test_idx],Y.iloc[test_idx])

  model = CatBoostClassifier(loss_function='MultiClass', 
                         eval_metric='TotalF1', 
                         depth=7,
                         n_estimators=5000,
                         verbose=0
)
  model.fit(*train_set)
  preds = model.predict_proba(test_set[0])
  score = log_loss(test_set[1],preds)
  
  oof_pred_fold = model.predict_proba(scaler.transform(test))
  oof_preds.append(oof_pred_fold)

  print(score)
  oof_scores.append(score)

print("Mean OOF Score :",np.mean(oof_scores))
final_preds = np.mean(np.array(oof_preds),axis=0)

0.3884213073422873
0.4087913810928163
0.40844541256610956
0.40156063740758347
0.3751284592841273
0.38386691912139076
0.3722101884776703
0.38647797701520153
0.37904923868074564
0.39205411191904815
Mean OOF Score : 0.38960056329069803


In [29]:
temp = pd.DataFrame(final_preds)
sub = pd.DataFrame()
sub['0'] = 0
sub['1'] = temp[0]
sub['2'] = temp[1]
sub['3'] = temp[2]
sub['4'] = temp[3]
sub = sub.fillna(0)

In [30]:
sub.head()

,0,1,2,3,4
0,0.0,0.009259,0.794837,0.155275,0.040629
1,0.0,0.003089,0.018439,0.977119,0.001353
2,0.0,0.000301,0.019935,0.972577,0.007187
3,0.0,0.001710,0.028403,0.959570,0.010318
4,0.0,0.000067,0.001331,0.997125,0.001477


In [31]:
np.sum(sub['0'])

0.0

In [32]:
#there are duplicates between test and train, for those rows we can directly use train target values
#storing the target values of common rows in test and train
temp1=''
for i in test.columns:
    temp1=temp1+i+train[i].astype(str)
train['ALL']=temp1
temp2=''
for i in test.columns:
    temp2=temp2+i+test[i].astype(str)
test['ALL']=temp2
encoding=train.groupby('ALL')['popularity'].median()
y_from_train=test['ALL'].map(encoding)
temp = pd.DataFrame()
temp['pop'] = y_from_train
temp['pop'] = temp['pop'].fillna(-999)
x = temp[temp['pop']!=-999]
for i in x.index.values:
#     print(x.iloc[i].values[0])
    try:
        if x.loc[i].values[0] ==0:
            sub.loc[i] =[1.0,0.0,0.0,0.0,0.0]
        elif x.loc[i].values[0] ==1:
            sub.loc[i] =[0.0,1.0,0.0,0.0,0.0]
        elif x.loc[i].values[0] ==3:
            sub.loc[i] =[0.0,0.0,1.0,0.0,0.0]
        elif x.loc[i].values[0] ==4:
            sub.loc[i] =[0.0,0.0,0.0,1.0,0.0]
        elif x.loc[i].values[0] ==5:
            sub.loc[i] =[0.0,0.0,0.0,0.0,1.0]
        else:
            print('.',end='')
    except:
        print(i)
        print(x.loc[i].values[0])

In [33]:
dummy = test[test['Store_Ratio']==0]
dummy.index
for i in dummy.index.values:
    sub.iloc[i] =[1.0,0.0,0.0,0.0,0.0]
sub.head()

,0,1,2,3,4
0,0.0,0.000000,1.000000,0.000000,0.000000
1,0.0,0.003089,0.018439,0.977119,0.001353
2,0.0,0.000301,0.019935,0.972577,0.007187
3,0.0,0.001710,0.028403,0.959570,0.010318
4,0.0,0.000000,0.000000,1.000000,0.000000


In [34]:
np.sum(sub['0'])

11.0

In [35]:
sub.to_csv('sub_2701_cat_fe_v3.csv',index=False)